# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
# from collections import Counter
import json
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv('main_task.csv')

In [3]:
df.sample(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
26603,id_651,Lyon,['French'],652.0,4.0,$$ - $$$,61.0,"[['Great wine and platter. Friendly staff', 'M...",/Restaurant_Review-g187265-d2718480-Reviews-Le...,d2718480
12752,id_222,Madrid,"['Mediterranean', 'European', 'Spanish', 'Vege...",223.0,4.5,$$ - $$$,128.0,"[['Best Jamon Iberico we had in Spain', 'Madri...",/Restaurant_Review-g187514-d11938372-Reviews-E...,d11938372
24855,id_13763,London,NaN,13773.0,4.5,NaN,8.0,"[['When in Petticoat Lane', 'Good coffee frien...",/Restaurant_Review-g186338-d5567535-Reviews-Ca...,d5567535
12360,id_13684,Paris,['Italian'],13686.0,5.0,NaN,2.0,"[['Very good food'], ['09/27/2017']]",/Restaurant_Review-g187147-d12571253-Reviews-M...,d12571253
25991,id_8071,Madrid,NaN,8078.0,3.0,NaN,21.0,"[[], []]",/Restaurant_Review-g187514-d1599930-Reviews-Ca...,d1599930


Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvosor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [ ]:
df.info()

In [ ]:
df[df.duplicated()]   # проверяем, есть ли дубли в данных

# Задания из модуля

In [ ]:
# 4.1
df['Price Range'].value_counts()

In [ ]:
# 4.2
df['City'].value_counts().count()

In [ ]:
# 4.3
print(df['Cuisine Style'].value_counts())
print(type(df['Cuisine Style'][0]))

pat = re.compile('\'(.*?)\'')
# cuisine_styles = df['Cuisine Style'].apply(lambda x: ['default'] if pd.isnull(x) else pat.findall(x))
cuisine_styles = df['Cuisine Style'].dropna().apply(lambda x: pat.findall(x))
cuisine_styles.explode().value_counts(ascending=False)

In [ ]:
# 4.4
def replacer(matches):
    """
    Defines how re.sub should format a string including special cases.
    """
    
#     След фиксы пока в виде костылей, возм исправлю позже
    text = re.sub('\\\\x7f', '', matches.group('texts'))   # удаляем один бинарный символ (срока ...)
    text = re.sub('\\\\', '', text)   # удаляем одиночный \ в слове "can\'t" (строка ...)
    text = re.sub('\sna', " 'nan", text)   # для строки 5315, содержащей nan
    text = re.sub('an,\s', "nan', ", text)   # то же для строки 22958, содержащей nan в начале
    
    text = re.sub('"', "'", text)   # для всех цитат и апострофов в отзывах заменяем " на ', тк json.loads не работает с "
    texts = re.sub('((\'|\"), (\'|\"))', '", "', text)
    dates = re.sub('((\'|\"), (\'|\"))', '", "', matches.group('dates'))
    
    return '[["' + texts + '"], ["' + dates + '"]]'


def parse_review(string, obj):
    """
    Converts strings of df['Reviews'] into json.loads digestiable format.
    """
    
    string = re.sub('(\[\[.)(?P<texts>.*?)(.\], \[.)(?P<dates>.*?)(.\]\])', replacer, string)
    lst = json.loads(string)   # парсим

    if obj == 'Reviews':
        return lst[0] or None
    else:
        return lst[1] or None


def compare_lengths(row):
    """
    Checks if parsed lists of according reviews and dates are of the same length.
    """
    
    if row['Review'] is not None:
        if len(row['Review']) != len(row['Review date']):
            display(row)
            print(len(row['Review']), len(row['Review date']))


reviews_df = pd.DataFrame(columns=['Review', 'Review date'])
reviews_df['Review'] = df['Reviews'].apply(parse_review, obj='Reviews')
reviews_df['Review date'] = df['Reviews'].apply(parse_review, obj='Dates')
display(reviews_df)

reviews_df.apply(compare_lengths, axis=1)
reviews_df = reviews_df.apply(pd.Series.explode)#.reset_index()
reviews_df['Review date'] = pd.to_datetime(reviews_df['Review date'])
display(reviews_df)

In [ ]:
# строки, на которых спотыкалось:
stumble1 = str([["Don't eat here if you have a train to catc...", 'Lunchtime drink'], ['10/11/2017', '07/03/2017']])
stumble2 = str([['"Best Local"', 'Perfect pitstop'], ['02/09/2017', '04/17/2015']])
stumble3 = str([['Lillian restaurant reviews', 'Greasy Spoon\x7f'], ['12/20/2017', '07/09/2017']])
stumble4 = str([['Sat night', 'Can\'t even explain.. "A" place.. A "100...'], ['10/22/2017', '05/11/2017']])
stumble5 = df.iloc[5315]['Reviews']   # 5315
stumble6 = df.iloc[22958]['Reviews']   # 22958


def replacer1(matches):
    text = re.sub('\\\\x7f', '', matches.group('texts'))   # удаляем один бинарный символ (срока ...)
    text = re.sub('\\\\', '', text)   # удаляем одиночный \ в слове "can\'t" (строка ...)
    text = re.sub('\sna', " 'nan", text)   # для строки 5315, содержащей nan в конце
    text = re.sub('an,\s', "nan', ", text)   # то же для строки 22958, содержащей nan в начале
    text = re.sub('"', "'", text)   # для всех цитат и апострофов в отзывах заменяем " на ', тк json.loads не работает с "
    texts = re.sub('((\'|\"), (\'|\"))', '", "', text)
    dates = re.sub('((\'|\"), (\'|\"))', '", "', matches.group('dates'))
    return '[["' + texts + '"], ["' + dates + '"]]'


# stumble3.encode('utf-8').replace(b'\x7f', '')
print(stumble3)
print(stumble3.replace(r'\x7f', ''))
print(stumble4.replace('\\', '\\\\'))

stumble5 = re.sub('(\[\[.)(?P<texts>.*?)(.\], \[.)(?P<dates>.*?)(.\]\])', replacer1, stumble5)
# lst1 = json.loads(stumble5)   # парсим
print(stumble5)

y = eval(stumble4)
type(y)

In [ ]:
# 4.4
print(reviews_df['Review date'].max())
last = reviews_df['Review date'].max()
first = reviews_df['Review date'].min()
print(last, first)
print(last - first)

В ходе выполнения заданий из модуля выяснили, что столбцы "Cuisine Style", "Reviews" заполнены строками, выглядящими как списки, превратили их в списки, проверили корректность заполнения, для "Cuisine Style" посчитали число уникальных значений, собрали их в массив; "Reviews" разбили на отзывы и их даты, поставили в соответствие. Т.о. сделана часть работы по извлечению инсайтов.

# Извлекаем инсайты

<i>Сategorical parameters:</i> Restaurant_id, City, Cuisine Style, Reviews, URL_TA, ID_TA

<i>Ordinal parameters:</i> Price Range

<i>Numerical parameters:</i> Ranking, Rating, Number of Reviews

## Categorical parameters

### 1. Restaurant_id

In [ ]:
print('Total input number: {}'.format(len(df['Restaurant_id'])))
print('Total amount of restaurants: {}'.format(df['Restaurant_id'].nunique()))
print('Most comon restaurants (over 10 inputs): ')
common_rests = df['Restaurant_id'].value_counts()
common_rests

In [ ]:
# check if restaurants with the same id are located in one city (or it's the same restaurant)
for rest in common_rests.index:
#     print('Rest {}, number of inputs: {}, number of cities: {}'.format(rest,     # вывод слишком длинный, поэтому см ниже
#             common_rests[rest], df[df['Restaurant_id'] == rest]['City'].nunique()))
    if common_rests[rest] != df[df['Restaurant_id'] == rest]['City'].nunique():
        print('inequal')

**Вывод:** один и тот же ресторан в разных городах выступает под одним id (возможно, сети). Записи о ресторанах не повторяются, пропусков нет.

### 2. City

In [ ]:
df['City'].value_counts()

Города указаны должным образом, пропусков нет, необходимо сгенерировать dummy-переменные.

### 3. Cuisine Style

Cм задание 4.3.

В этой колонке содержатся строки, визуально выглядящие как списки. Есть пропуски, попытаемся заполнить их ниже.

### 4. Reviews

См. задание 4.4.

Также строки, выглядящие как списки, каждая содержит 2 "списка" - первый с отзывами, второй с их датами, по 0-2 отзыва в каждой ячейке. Разделим их и вынесем в отдельные колонки.

Есть как настоящие пропуски, так и пустые списки. Поэтому после извлечения настоящих отзывов получится немало пропусков.

In [ ]:
print(df['Reviews'][1], '\n', type(df['Reviews'][0]))
print(df['Reviews'][0])
print('Кол-во пропусков: {}'.format(df['Reviews'].isna().sum()))

In [ ]:
df['Reviews_new'] = df['Reviews'].apply(parse_review, obj='Reviews')
df['Review_dates'] = df['Reviews'].apply(parse_review, obj='Dates')

In [ ]:
df.sample(3)

In [ ]:
def extract_item(cell):
    if cell and i < len(cell):
        return cell[i]
    else:
        return None


for i in range(2):
    col_name = 'Review ' + str(i + 1)
    df[col_name] = df['Reviews_new'].apply(extract_item)
    
    col_name_date = 'Date ' + str(i + 1)
    df[col_name_date] = pd.to_datetime(df['Review_dates'].apply(extract_item))

In [ ]:
# насколько полно заполнены отзывы и их даты:
df.info()

In [ ]:
df.drop(['Reviews', 'Reviews_new', 'Review_dates'], axis=1, inplace=True)

In [ ]:
df.sample(3)

### 5. URL_TA

In [ ]:
df['URL_TA'].nunique()

Число уникальных записей на 20 меньше, чем общее число записей. Посмотрим на повторяющиеся.

In [ ]:
df['URL_TA'].value_counts()

Видно, что в ссылке на ресторан на TripAdvisor "зашиты" названия и местонахождения (геолокации) ресторанов, а также какие-то численные данные с префиксами d и g. Эти признаки можно из ссылок извлечь, интерпретировать и использовать в дальнейшем. Например, по названию ресторана можно попробовать определить кухню, которую в нем предлагают. Геолокация вряд ли понадобится, т.к. колонка City у нас не содержит пропусков.

Можно проверить, совпадают ли города в колонке City с городами в ссылке.

Извлекаем признаки:

In [ ]:
def find_pat(cell):
    match = pat.search(cell)
    return match.group(feature) if match else None


pat = re.compile('(?P<URL_g>g.*)-(?P<URL_d>d.+)-Reviews-(?P<Restaurant_title>.*)-(?P<Restaurant_geo>.*)\.')
re_groups = ['URL_g', 'URL_d', 'Restaurant_title', 'Restaurant_geo']


for feature in re_groups:
    df[feature] = df['URL_TA'].apply(find_pat)

In [ ]:
df.sample(5)

In [ ]:
# проверим те строки, где не был найден паттерн (regex не сработал):
subset = df[df['URL_g'].isna()]
display(subset)
for i in subset['URL_TA']:
    print(i)

In [ ]:
# воспользуемся отдельным паттерном для этих строк:
inds = subset.index

pat1 = re.compile('(?P<URL_g>g.*)-(?P<URL_d>d.+)-Reviews-(?P<Restaurant_geo>.*)\.')
re_groups1 = ['URL_g', 'URL_d', 'Restaurant_geo']

for feature in re_groups1:
    df[feature][inds] = df['URL_TA'][inds].apply(lambda x: pat1.search(x).group(feature))

In [ ]:
# проверим, все ли заполнилось:
df.info()

Теперь посмотрим, совпадают ли города в колонке City с городами, извлеченными из ссылок на рестораны в TripAdvisor. Заодно возьмем на рассмотрение те рестораны, где кухни отсутствовали:

In [ ]:
df[df['Cuisine Style'].isna()].sample(5)

Гео, извлеченное из URL, содержит более развернутую информацию о местонахождении ресторана (район города, муниципальный округ и т.д.).

### 5. URL_g

In [ ]:
print(df['URL_g'].nunique())
df['URL_g'].value_counts()

In [ ]:
print(df[df['URL_g'] == 'g186338'].shape)
df[df['URL_g'] == 'g186338'].sample(5)

Скорее всего, этот признак соответствует геолокации ресторана, т.е. совпадает с 'Restaurant geo' (не с 'City' - ранее видели, что всего городов 31, т.е. меньше, чем никальных значений 'URL_g').

In [ ]:
print(df['Restaurant_geo'].nunique())
df['Restaurant_geo'].value_counts()

Не совсем - цифры различаются. Далее углубляться не будем, все равно колонка с городами не содержит пропусков - использовать URL_g незачем.

### 6. URL_d

In [ ]:
print(df['URL_d'].nunique())
df['URL_d'].value_counts()

Этот признак дублирует ID_TA (id ресторана на TripAdvisor-e) - см. ниже.

### 4. ID_TA

In [ ]:
df['ID_TA'].nunique()

In [ ]:
v_c = df['ID_TA'].value_counts()
v_c

Опять число уникальных записей на 20 меньше, чем общее число записей - как в URL_TA. Скорее всего, повторения этих признаков перекрываются. Посмотрим, что общего у записей с одинаковым ID_TA.

In [ ]:
rep_id_ta = v_c[v_c > 1].index
id_ta_subset = df[df['ID_TA'].isin(rep_id_ta)].sort_values('ID_TA')
display(id_ta_subset)
print('N of lines: {}'.format(id_ta_subset.shape[0]))

for col in id_ta_subset.columns:
    print('N unique in col {}: {}'.format(col, id_ta_subset[col].nunique()))

Все признаки одинаковы у пар ресторанов за исключением id, Ranking - они уникальны, однако очень близки. Возможно, произошел какой-то сбой и были внесены записи об одних и тех же ресторанах, только с небольшой разницей во времени.

In [ ]:
# подумать, как можно заполнить пропуски в этой выборке

In [ ]:
df.sample(5)

## Ordinal parameters

In [ ]:
print(df['Price Range'].isna().sum())
df['Price Range'].value_counts()

Колонка заполнена корректно, можно заменить на численные эквиваленты значений (см ниже в feature engineering).

Имеются пропуски, ниже попытаемся заполнить на основе связей с другими признаками.

## Numerical parameters

In [ ]:
df[['Ranking', 'Rating']].corr()

In [ ]:
df[['Ranking', 'Rating']].hist()

In [ ]:
# sns.boxplot(data=df[['Ranking', 'Rating']], hue=df[['Ranking', 'Rating']].columns)
sns.boxplot(data=df['Ranking'])

# Заполнение пропусков

Т.о. пропуски содержатся в колонках **'Cuisine Style', 'Price Range', 'Number of Reviews', 'Review n', 'Date n', 'Restaurant title'**.

1. **'Cuisine Style', 'Price Range', 'Number of Reviews'** можно попробовать заполнить на основе предполагаемых связей с другими признаками:

-- 'Cuisine Style': ресторан может принадлежать к сети, в сетевых ресторанах, как правило, подают одну и ту же кухню - проверить можно по названию 'Restaurant_title';

-- название ресторана 'Restaurant_title' может содержать или подразумевать кухню, которую в нем подают, например, ресторан с японским названием, расположенный в Италии, скорее всего предлагает японскую кухню;

-- 'Price Range' может также зависеть от принадлежности ресторана к сети;

-- его можно заполнить на основе распределения цен в ресторанах в каждом отдельном городе / подающих одну кухню / имеющих близкое число отзывов - в общем или в каждом отдельном городе / близкий 'Ranking'/'Rating' аналогично. Для этого скорее всего нужно формулировать гипотезы и статистически их проверять, затем заполнять пропуски на основе подтвержденных.

-- 'Number of Reviews' - на основе его корреляции с 'Ranking' в каждом отдельном городе.

2. **'Review n', 'Date n', 'Restaurant title'** заполнить невозможно.

<br>
<i>А можно не строить гипотез и спарсить данные прямо с сайта. Так и сделаем.</i>

In [7]:
df_parsed0 = pd.read_csv('parsed_data_output0.csv', index_col=0)
df_parsed0

,Restaurant_id,Cuisine Style,Price Range,Number of Reviews
19,id_2791,"['Chinese', 'Asian']",$$ - $$$,18.0
32,id_1327,NaN,NaN,1.0
13,id_4637,NaN,$,11.0
33,id_2434,['Vietnamese'],NaN,4.0
37,id_5209,"['Chinese', 'Japanese', 'Seafood']",$$ - $$$,74.0
...,...,...,...,...
9962,id_1540,"['Bar', 'European']",$$ - $$$,253.0
9973,id_6048,NaN,NaN,NaN
9970,id_760,"['Bar', 'European', 'Czech']",$,80.0
9952,id_2268,"['Asian', 'Thai', 'Vegetarian Friendly']",$,23.0


In [34]:
df_parsed0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9887 entries, 19 to 9936
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      9887 non-null   object 
 1   Cuisine Style      7894 non-null   object 
 2   Price Range        7337 non-null   object 
 3   Number of Reviews  8862 non-null   float64
dtypes: float64(1), object(3)
memory usage: 706.2+ KB


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [36]:
df_parsed0['Restaurant_id'].nunique()

5946

In [37]:
df.iloc[:10000]['Restaurant_id'].nunique()

5988

In [54]:
for idx, row in df.dropna(axis=0, subset=['Cuisine Style']).iterrows():
    rest_id = row['Restaurant_id']
    current_cuisines = df_parsed0.loc[df_parsed0['Restaurant_id'] == rest_id, 'Cuisine Style']
    if len(current_cuisines) > 1: # and current_cuisines.notna().sum() > 0:
        print('over 1\n', current_cuisines)
        df.loc[idx, 'Cuisine Style'] = [current_cuisines.loc[current_cuisines.str.len().idxmax()] or None]
        # пока выбираем самую длинную строчку с кухнями
        # можно вместо этого выбирать самые встречающиеся кухни из полученного набора для данного id
    else:
#         print('not over 1\n', current_cuisines)
#         print(current_cuisines.iloc[0])
        df.loc[idx, 'Cuisine Style'] = [current_cuisines.iloc[0]]

not over 1
 0    ['French', 'International', 'European']
Name: Cuisine Style, dtype: object
['French', 'International', 'European']
over 1
 2       ['Japanese', 'Sushi', 'Asian']
9162                               NaN
Name: Cuisine Style, dtype: object
over 1
 4              ['German', 'European', 'Central European']
485             ['Bar', 'International', 'Mediterranean']
4484    ['Mediterranean', 'Greek', 'Vegetarian Friendly']
5319                                                  NaN
Name: Cuisine Style, dtype: object
over 1
 6                         ['Italian', 'Pizza', 'Seafood']
6787    ['Mediterranean', 'Spanish', 'Vegetarian Frien...
Name: Cuisine Style, dtype: object
over 1
 7                                ['Italian']
1044         ['Chinese', 'Seafood', 'Asian']
1448                  ['Polish', 'European']
2448    ['Quick Bites', 'Cafe', 'Fast Food']
5423                                ['Cafe']
Name: Cuisine Style, dtype: object
not over 1
 9    ['Italian', 'Pizza', 'Fast F

 94                  ['Bar', 'British', 'Pub']
8210        ['Quick Bites', 'Bar', 'British']
9988    ['Italian', 'Pizza', 'Mediterranean']
Name: Cuisine Style, dtype: object
not over 1
 95    ['French', 'Gastropub']
Name: Cuisine Style, dtype: object
['French', 'Gastropub']
over 1
 97      ['Cafe', 'Healthy', 'Vegetarian Friendly']
303              ['American', 'Steakhouse', 'Bar']
2307                 ['Lebanese', 'Mediterranean']
3679        ['European', 'Scandinavian', 'Danish']
5461       ['Italian', 'Seafood', 'Mediterranean']
6436      ['Seafood', 'Mediterranean', 'European']
8958                                    ['French']
Name: Cuisine Style, dtype: object
over 1
 98                ['Italian', 'Pizza', 'Mediterranean']
2283                                                NaN
6618              ['Italian', 'Pizza', 'Mediterranean']
7825    ['European', 'Wine Bar', 'Vegetarian Friendly']
Name: Cuisine Style, dtype: object
over 1
 99      ['Chinese', 'Asian', 'Vegetarian Friendly'

TypeError: reduction operation 'argmax' not allowed for this dtype

In [52]:
rest_id = df.iloc[100]['Restaurant_id']
display(df_parsed0.loc[df_parsed0['Restaurant_id'] == rest_id])
current_cuisines = df_parsed0.loc[df_parsed0['Restaurant_id'] == rest_id, 'Cuisine Style']
print(current_cuisines)
if len(current_cuisines) == 1:
    print(current_cuisines.values)
#     print(current_cuisines.str.len().idxmax())
#     print(current_cuisines.loc[current_cuisines.str.len().idxmax()])
#     print(type(current_cuisines.loc[current_cuisines.str.len().idxmax()]))

,Restaurant_id,Cuisine Style,Price Range,Number of Reviews
100,id_2904,['Italian'],NaN,4.0
2353,id_2904,"['Italian', 'Mediterranean', 'Spanish']",$,29.0
3712,id_2904,"['German', 'European', 'Vegetarian Friendly']",$$ - $$$,35.0
4537,id_2904,['Mexican'],$,32.0
7460,id_2904,"['Chinese', 'Asian', 'Cantonese']",$$ - $$$,21.0


100                                       ['Italian']
2353          ['Italian', 'Mediterranean', 'Spanish']
3712    ['German', 'European', 'Vegetarian Friendly']
4537                                      ['Mexican']
7460                ['Chinese', 'Asian', 'Cantonese']
Name: Cuisine Style, dtype: object


In [20]:
df.loc[9962, 'Cuisine Style']

"['American', 'Bar', 'European', 'Pub']"

In [25]:
df.loc[9962, ['Restaurant_id', 'Cuisine Style', 'Price Range', 'Number of Reviews']]

Restaurant_id                                       id_1540
Cuisine Style        ['American', 'Bar', 'European', 'Pub']
Price Range                                        $$ - $$$
Number of Reviews                                       147
Name: 9962, dtype: object

In [43]:
df_parsed0['Cuisine Style'].str.len().idxmax()

3180

### 1. Cuisine Style

In [ ]:
# проверяем, много ли пропусков. Ранее мы видели, что id ресторана не зависит от принадлежности к сети, т.е. у одинаковых
# id могут быть совершенно разные кухни. Проверим зависимость от названия, извлеченного из url:
print('Число пропусков в кухнях: {}'.format(df['Cuisine Style'].isna().sum()))
print('Число уникальных названий ресторанов в строках с пропущенными кухнями: {}'.format(df[df['Cuisine Style'].isna()]
                                                                                         ['Restaurant_title'].nunique()))

In [ ]:
df['Restaurant_title'].value_counts()

In [ ]:
print(df[df['Restaurant_title'] == 'Burger_King'].shape)
df[df['Restaurant_title'] == 'Burger_King'].sample(5)

In [ ]:
print(df[df['Restaurant_title'] == 'Hippopotamus'].shape)
df[df['Restaurant_title'] == 'Hippopotamus'].sample(5)

Т.о. имеются сети ресторанов с пропусками в кухнях => их можно легко заполнить.

NB: Стили кухонь могут немного отличаться в зависимости от геолокации. Можно заполнять в зависимости от гео, однако не будем усложнять задачу и заполним наиболее популярными значениями из выборок по названиям ресторанов.

In [ ]:
# функция очень долгая, необходимо оптимизировать
# заполняем пропуски кухонь в сетевых ресторанах

def fill_cuisine_na(row):
    """
    Fills in missing cuisine style for chain restaurants.
    """
    
    if pd.isnull(row['Cuisine Style']):
        title = row['Restaurant_title']
        if title:   # проверяем, нет ли пропуска в названии ресторана, у нас имеются 2 таких строки. Если пропуск - вернется None
            subset = df[df['Restaurant_title'] == title]['Cuisine Style']
            if subset.nunique() > 0:                    # если наборы кухонь разные
                return subset.value_counts().idxmax()   # возвращаем наиболее часто встречающееся значение
    else:
        return row['Cuisine Style']


cuisine_styles_filled = df.apply(fill_cuisine_na, axis=1)

In [ ]:
cuisine_styles_filled.isna().sum()   # сократили на ~1000

In [ ]:
# сначала преобразуем строки в списки
# pat = re.compile('\'(.*?)\'')
# df['Cuisine Style'] = df['Cuisine Style'].dropna().apply(lambda x: pat.findall(x))   # regex is more efficient that eval()

In [ ]:
# заполняем пропуски кухонь на основе названий ресторанов:

# def cuisine_from_title(row):
#     title = ...
#     for word in df[]:
        
def split_title(title):
    if pd.isnull(title):
        return None
    else:
        splitted = title.split('_')
        splitted = list(filter(lambda x: len(x) > 2, splitted))
        return splitted

titles_words = df[df['Cuisine Style'].notna()]['Restaurant_title'].apply(split_title)
titles_of_na_cuisines = df[df['Cuisine Style'].isna()]['Restaurant_title'].apply(split_title)

In [ ]:
titles_words

In [ ]:
print(titles_of_na_cuisines.iloc[2])

cur_cuisines = []

for idx, title in np.ndenumerate(df[df['Cuisine Style'].notna()]['Restaurant_title'].dropna()):
    if any(word in title for word in titles_of_na_cuisines.iloc[2]):
        print(title)
        print(df.iloc[idx]['Cuisine Style'])
        cur_cuisines.append(df.iloc[idx]['Cuisine Style'])

In [ ]:
df['Restaurant_title'][:10]

In [ ]:
# на основе распределения кухонь по городам - последний и наименее точный метод:

def cuisine_distr_in_city(city):
    cuisines = df[df['Restaurant_geo'] == city]['Cuisine Style'].apply(lambda x: None if pd.isnull(x)
                                                        else re.findall('\'(.*?)\'', x)).explode().value_counts()
    print(city)
    print(cuisines.nlargest(3))
    
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.barplot(x=cuisines.index, y=cuisines.values, data=cuisines, ax=ax)
    plt.show()


na_cuisine_geos = df[df['Cuisine Style'].isna()]['Restaurant_geo'].unique()   # города, в которых есть пропуски кухонь
# df[df['Restaurant_geo'] == cui_na_geos[0]]['Cuisine Style'].apply(lambda x: None if pd.isnull(x) else re.findall('\'(.*?)\'', x)).explode().value_counts().plot.bar()

for city in na_cuisine_geos[:10]:
    cuisine_distr_in_city(city)

In [ ]:
df.sample(3)

In [ ]:
df['Cuisine Style'] = cuisine_styles_filled

### 2. URL_g

Из url мы извлекли геолокацию ресторанов, проверим, есть ли пустоты в городах и можно ли их заполнить с помощью нового параметра.

In [ ]:
df['URL_g'].value_counts()

In [ ]:
df[df['City'].isna()].sort_values('URL_g')

# Feature engineering

## Categorical parameters

### 1. Cuisine Style

In [ ]:
# pd.concat([df[['Restaurant_id']], pd.get_dummies(df.explode('Cuisine Style')['Cuisine Style'], prefix='Cuisine', dummy_na=True)], axis=1, verify_integrity=True).drop_duplicates()

In [ ]:
# option: explode by cuisines, get dummies, drop duplicates

def cuisine_dummies(rest):
#     print(rest, type(rest))
    if pd.isnull(rest):   # if cuisines in strings
#     if isinstance(rest, list) is False:   # if cuisines in lists
        return 0
    else:
        return int(cuisine in rest)     
    
    
unique_cuisine_styles = np.sort(cuisine_styles.explode().unique())

for cuisine in unique_cuisine_styles:
    cuisine_prefixed = 'Cuisine_' + cuisine
    df[cuisine_prefixed] = df['Cuisine Style'].apply(cuisine_dummies)

df['Cuisine_nan'] = df['Cuisine Style'].apply(lambda x: 1 if pd.isnull(x) else 0)   # if cuisines in strings
# df['Cuisine_nan'] = df['Cuisine Style'].apply(lambda x: 0 if isinstance(x, list) else 1)   # if cuisines in lists

df.sample(5)

In [ ]:
# from stackoverflow:
# https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies
# pd.get_dummies(df['Cuisine Style'].apply(pd.Series).stack(), dummy_na=True).sum(level=0)

### 2. City

In [ ]:
df = pd.concat([df, pd.get_dummies(df['City'], prefix='City', dummy_na=True)], axis=1, verify_integrity=True)
df.sample(5)

### 3. Reviews

In [ ]:
# ver 1.0
# df['Review'] = df['Reviews'].apply(parse_review, obj='Reviews')
# df = df.explode('Review')
# df.loc[:, ['Review', 'Review date']] = reviews_df.loc[:, ['Review', 'Review date']]

# # feature engineering
# df['Review year'] = df['Review date'].dt.year
# # df['Review month'] = df['Review date'].dt.month
# # df['Review day'] = df['Review date'].dt.day
# # df['Review day of week'] = df['Review date'].dt.dayofweek   # без них результат лучше
# df['Review day of year'] = df['Review date'].dt.dayofyear

# df.drop('Reviews', axis=1, inplace=True)
# df.reset_index(inplace=True, drop=True)
# df.sample(5)

In [ ]:
# ver 2.0:
# найдем разницу во времени между отзывами:

def get_time_gap(row):
    return (row['Date 1'] - row['Date 2']).days if row['Date 1'] and row['Date 2'] else None


df['Time gap'] = df.apply(get_time_gap, axis=1)

In [ ]:
# насколько стары отзывы:

def review_age(cell):
    return (now - cell).days if cell else None


now = datetime.now()
df['Review 1 age'] = df['Date 1'].apply(review_age)
df['Review 2 age'] = df['Date 2'].apply(review_age)

In [ ]:
# какой день в году:
df['Review 1 day of year'] = df['Date 1'].dt.dayofyear
df['Review 2 day of year'] = df['Date 2'].dt.dayofyear

In [ ]:
df.sample(3)

In [ ]:
# длины отзывов (число слов):
df['Review 1 length'] = df['Review 1'].apply(lambda x: len(x.split()) if x else None)
df['Review 2 length'] = df['Review 2'].apply(lambda x: len(x.split()) if x else None)

In [ ]:
# отзывы:
df[['Review 1', 'Review 2']].dropna().stack().value_counts()[47700:]

# каждое слово в отдельности:
# df[['Review 1', 'Review 2']].dropna().stack().apply(lambda x: x.split()).explode().apply(lambda x: x.lower() if len(x)>2 else None).value_counts()[30:60]

In [ ]:
# оцифровываем отзывы: переведем оценочные суждения в отзывах в систему оценок. Т.к. нлп мы еще не владеем, проведем вручную:
impr_dict = {
    5: ['excellent', 'delicious', 'great', 'wonderful', 'very good', 'amazing', 'yummy', 'fantastic', 'lovely',
       'perfect', 'very nice', 'awesome', 'hidden gem', 'little gem', 'superb', 'brilliant', 'wow', 'loved', 'love', 
       'recommended', 'outstanding', 'fabulous', 'yum', 'delightful', 'surprise', 'surprised', 'best', 'enjoyable',
       'cool', 'really good', 'super', 'what a find', 'unique', 'charming', 'heaven'],
    4: ['good', 'not bad', 'nice', 'tasty', 'worth a visit', 'cheerful', 'pretty good', 'friendly', 'fine', 
        'better than expected', 'cozy', 'reasonable'],
    3: ['average', 'ok', 'nothing special', 'overpriced', 'not great', 'expensive', 'overrated', 'hit and miss',
       'not impressed', 'not too bad', 'mixed feelings', 'meh ', 'typical', 'standard'],
    2: ['disappointing', 'disappointed', 'bad', 'poor', 'below average', 'disappointment', 'rude', 'waste', 'wasting',
       'underwhelmed'],
    1: ['terrible', 'awful', 'avoid', 'horrible', 'very bad', 'stay away', 'rip off', 'disgusting', 'worst', 'closed']
}

impr_df = pd.DataFrame(dict([(k,pd.Series(v)) for k, v in impr_dict.items()]))

# def get_review_rate(cell):
#     for word in impr_df.values:
#         print(word, type(word))
# #         return ... if word in cell.lower() else None
#         if word in cell.lower():
#             print(impr_df[impr_df == word].columns)


def get_review_rate(cell):
    if cell:
        for mark in impr_dict:
            for word in impr_dict[mark]:
                if word in cell.lower():
                    return mark


df['Review 1 rate'] = df['Review 1'].apply(get_review_rate)
df['Review 2 rate'] = df['Review 2'].apply(get_review_rate)

In [ ]:
df.sample(3)

In [ ]:
df[['Review 1 length', 'Review 2 length', 'Review 1', 'Review 2']].info()

## Ordinal parameters

In [ ]:
price_ranges = {np.nan: 0, '$': 1, '$$ - $$$': 2, '$$$$': 3}
df['Price Range'] = df['Price Range'].apply(lambda x: price_ranges[x])

In [ ]:
df.sample(3)

### Заполнение пропусков в Price Range

In [ ]:
df[df['Restaurant_geo'] == 'Barcelona_Catalonia']['Price Range'].hist()

In [ ]:
df[(df['Restaurant_geo'] == 'Barcelona_Catalonia') & (df['Cuisine Style'] == "['Mediterranean']")]['Price Range'].hist()

In [ ]:
df['Restaurant_geo'].value_counts()

In [ ]:
df[df['Restaurant_geo'] == 'Barcelona_Catalonia']['Cuisine Style'].value_counts()

Заполнять пропуски в этой колонке нужно осмысленно, значения зависят от разных факторов, например одновременно от города и стиля кухни - видно по разному характеру графиков, когда во внимание берется один или два параметра.

Доделаю эту работу похже.

## Удаляем текстовые признаки

In [ ]:
# v 2.0 for ML
# obj_cols = ['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'URL_TA', 'ID_TA', 'Review', 'Review date']
# obj_cols1 = ['City', 'Cuisine Style', 'Price Range', 'URL_TA', 'ID_TA', 'Review', 'Review date']
obj_cols2 = ['City', 'Cuisine Style', 'ID_TA', 'Review 1', 'Review 2', 'Date 1', 'Date 2', 'URL_TA', 'URL_g', 'URL_d', 'Restaurant_title', 'Restaurant_geo']
df = df.drop(obj_cols2, axis=1)

# values = {col: df[col].mean() for col in 
#           ['Ranking', 'Rating', 'Number of Reviews', 'Review year', 'Review month', 'Review day']}
# values = {col: df[col].mean() for col in 
#           ['Ranking', 'Rating', 'Number of Reviews', 'Review year', 'Review day of year']}
# print(values)
df.fillna(df.median(), inplace=True)   # filling in with means

df.sample(5)

In [ ]:
df.fillna(df.median(), inplace=True)

In [ ]:
df.sample(3)

In [ ]:
# df.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
# v 1.0 for ML
# df = pd.read_csv('main_task.csv')

# # obj_cols = ['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA']
# obj_cols = ['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA']

# df = df.drop(col_drop, axis=1)
# # df.fillna(0, inplace=True)

# values = {col: df[col].mean() for col in df.columns}
# df.fillna(values, inplace=True)

# df.sample(5)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
# X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
# X = df.drop(['Rating'], axis = 1)
# y = df['Rating']
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

# Записываем результат для сабмита

In [ ]:
result = pd.DataFrame(df.iloc[X_test.index]['Restaurant_id'])
result['Rating'] = y_pred
result['Rest_id'] = result['Restaurant_id'].apply(lambda x: int(x[3:]))
result = result.sort_values('Rest_id')#.drop_duplicates(subset=['Restaurant_id'])
result = result.drop('Rest_id', axis=1)
result.head(20)

In [ ]:
result.info()

In [ ]:
submit_time = datetime.now().strftime('%Y.%m.%d %H-%M')
filename = submit_time + ' rds3_Walrave.csv'
result.to_csv(path_or_buf=filename, index=False)

In [ ]:
pd.read_csv(filename)